# Human Activity Recognition using Clustering


**Integrantes:**
- Lucas Carranza
- Guillermo Sánchez
- David Herencia
- José Osnayo

In [4]:
import numpy as np
import pandas as pd

## Dataset Analysis

In [7]:
print("Hola mundo")

Hola mundo


## Reducción de Dimensionalidad

## Implementación

La implementación de los Modelos de Clustering a usar

### Modelo 1

### Modelo 2

## Submission

Archivo .csv para la competencia de Kaggle: https://www.kaggle.com/competitions/project-3-clustering